# **Instruções Gerais**

- **Copie** este notebook, junto com as imagens da pasta para o seu Google Drive da UFV
- Resolva o que está sendo pedido no próprio notebook
- Lembre-se de **montar o seu Google Drive** no sistema de arquivos do notebook (ícone da pastinha, na barra de ferramentas à esquerda)
- Entregue, como resposta à atividade no **Moodle**

- **Obs.:**
  - Lembre-se de **salvar** o notebook de tempos em tempos
  - Envie **apenas** o notebook. Não há necessidade de enviar as imagens

# **1. Preâmbulo**

## **1.1. Identifique-se**

Complete o que está sendo pedido na célula a seguir

In [ ]:
# Informe, dentro das strings, seu nome e número de matrícula
estudante = 'Erick Lima Figueiredo'
matricula = '98898'

## **1.2. Importações de módulos Python**

- As bibliotecas necessárias para a atividade já se encontram importadas na célula a seguir. Mas você pode adicionar novas importações, se achar necessário

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from skimage import color
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

- Altere o caminho abaixo para a pasta onde vc copiou os arquivos

In [ ]:
os.chdir('/content/drive/MyDrive/Praticas PDI')

## **1.3. Funções auxiliares já implementadas**

In [ ]:
def exibe_mosaico(imagens, legendas, colunas=4, arquivo=None):
  # Largura e altura básicos, de referência, das imagens
  l = 8
  a = 6

  n_imagens = len(imagens)
  if len(imagens) < colunas:
    colunas = len(imagens)
  linhas = n_imagens // colunas + int(n_imagens % colunas > 0)

  fig = plt.figure(figsize=(l*colunas,a*linhas))
  ax = []

  i = j = 0
  for pos, img in enumerate(imagens):
    i = pos // colunas
    j = pos % colunas
    ax.append(fig.add_subplot(linhas, colunas, pos+1))
    ax[-1].axis('off')
    if img.mode == 'L' or img.mode == '1':
      img = img.convert('RGB')
    arr = np.asarray(img)
    ax[-1].imshow(arr)
    ax[-1].set_title(legendas[pos])

  fig.tight_layout()

  if arquivo is not None:
    plt.savefig(arquivo, dpi=300)

  plt.show()

In [ ]:
def gera_imagem(arr):
  arrImg = arr.round().clip(0,255).astype(np.uint8)
  return Image.fromarray(arrImg)

In [ ]:
def cinza_lum(img, rgb=False, ret_img=False):
  arr = np.asarray(img).astype(float)

  T = [0.299, 0.587, 0.114]

  T = np.array(T).T

  arrC = np.dot(arr, T)

  if rgb:
    arrC = np.dstack((arrC, arrC, arrC))

  if ret_img:
    return gera_imagem(arrC)
  else:
    return arrC

# **2. Exercícios**

## **2.1. Binarização utilizando histograma CDF**

Implemente, no corpo da função abaixo, a técnica de binarização utilizando histograma CDF, vista em sala (slides 37 a 41).

A função deve retornar uma imagem pseud-binária (`PIL Image`), isto é, ao invés de definir os pixels com valores $0$ e $1$, utiliza o padrão: pixel preto: valor $0$ e pixel branco: valor $255$.

Quando binarizar, utilize o padrão onde os pixels pretos correspondem a valores **menores ou iguais** ao limiar de binarização.

In [ ]:
# tipos: 'b' --> básico, 'p' --> probabilidades, 'c' --> cdf
def gen_histogram(img, tipo='p'):
  arr = np.asarray(img).astype(int)

  arrCinza = np.dot(arr, np.array([0.299, 0.587, 0.144]).T)
  arrCinza = arrCinza.round().astype(int).clip(0,255) # clip poda valores dentro dos limites

  # Histograma básico
  H = np.zeros((4, 256)) # 3 canais + cinza x 256 níveis de intensidade

  for k in range(256):
    for c in range(3):
      H[c,k] = (arr[:,:,c] == k).sum()
    H[3,k] = (arrCinza == k).sum() # Contagem no nível k em cinza

  if tipo == 'b':
    return H

  # Histograma de probabilidades
  H /= arr.shape[0]*arr.shape[1]

  if tipo == 'p':
    return H

  # Histograma de probabilidades acumuladas (CDF)
  CDF = np.zeros_like(H)
  for k in range(256):
    soma = np.sum(H[:, :k+1], axis=1)
    CDF[:,k] = soma

  # Alternativa:
  # CDF = np.cumsum(H, axis=1)

  return CDF

In [ ]:
# img: imagem de entrada
# perc: percentual de pixels pretos
def bin_cdf(img, perc=0.5):
  arr = np.asarray(img).astype(int)

  arrCinza = np.dot(arr, np.array([0.299, 0.587, 0.144]).T)
  arrCinza = arrCinza.round().astype(int).clip(0,255)

  CDF = gen_histogram(img, 'c')

  threshold = np.abs(CDF[-1] - perc).argmin()

  arrCinza[arrCinza > threshold] = 255
  arrCinza[arrCinza <= threshold] = 0

  return gera_imagem(np.dstack([arrCinza, arrCinza, arrCinza]))


#### **Teste seu código**

Execute as 3 células a seguir

In [ ]:
img = Image.open('pimentas.png').convert('RGB')

imgs = [img]
legs = ['Original']

percs = np.linspace(0.1, 0.9, 7, endpoint=True)
for perc in percs:
  imgBin = bin_cdf(img, perc)

  imgs.append(imgBin)
  legs.append('CDF <= %.3f' % perc)

exibe_mosaico(imgs, legs)

In [ ]:
img = Image.open('por_do_sol.png')

imgs = [img]
legs = ['Original']

percs = np.linspace(0.1, 0.9, 7, endpoint=True)
for perc in percs:
  imgBin = bin_cdf(img, perc)

  imgs.append(imgBin)
  legs.append('CDF <= %.3f' % perc)

exibe_mosaico(imgs, legs)

In [ ]:
img = Image.open('moca.jpg')

imgs = [img]
legs = ['Original']

percs = np.linspace(0.1, 0.9, 7, endpoint=True)
for perc in percs:
  imgBin = bin_cdf(img, perc)

  imgs.append(imgBin)
  legs.append('CDF <= %.3f' % perc)

exibe_mosaico(imgs, legs)

## **2.2. Transferência de estilo alternativa**

Vimos em sala a implementação da transferência de estilo de cores entre imagens usando a Especificação Direta de Histogramas, cujo código já se encontra parcialmente implementado na célula a seguir.

Para que a função `especifica_histograma` funcione, no entanto, é necessário que uma outra função, que calcula o histograma CDF (`histogramaCDF`), esteja implementada. Implemente, portanto, o corpo da função `histogramaCDF` a seguir. Observe o código da função de especificação direta de histograma para deduzir qual e como deve ser a forma da estrutura de retorno que você deve implementar.

In [ ]:
def histogramaCDF(img):
  return np.delete(gen_histogram(img, 'c'), -1, axis = 0)


def especifica_histograma(imgForma, imgEstilo):
  # Calcula os histogramas CDF
  Cf = histogramaCDF(imgForma)
  Ce = histogramaCDF(imgEstilo)

  mapa = np.zeros((3, 256))
  for c in range(3):
    for k in range(256):
      p = Cf[c,k]
      diff = np.abs(Ce[c] - p)
      mapa[c,k] = np.argmin(diff)

  arrF = np.asarray(imgForma).astype(int)
  arrEsp = np.zeros_like(arrF)

  for i in range(arrF.shape[0]):
    for j in range(arrF.shape[1]):
      r,g,b = arrF[i,j]
      arrEsp[i,j] = [mapa[0,r], mapa[1,g], mapa[2,b]]

  return gera_imagem(arrEsp)

#### **Teste seu código parcial**

Execute a célula a seguir. A resposta esperada estará no notebook utilizado na aula.

In [ ]:
imgG = Image.open('grilo.jpg')
imgC = Image.open('capivara.jpg')

imgEspGC = especifica_histograma(imgG, imgC)
imgEspCG = especifica_histograma(imgC, imgG)

imgs = [imgG, imgC, imgEspGC, imgEspCG]
legs = ['Grilo', 'Capivara', 'Grilo <-- Capivara', 'Capivara <-- Grilo']
exibe_mosaico(imgs, legs)

#### **Técnica alternativa**

Vamos, agora, abordar uma técnica de transferência de estilos alternativa à especificação direta de histogramas.

Não há um nome oficial ou usual para a técnica, então a chamaremos aqui de *fast transfer*.

consiste em usar o desvio padrão e a média das cores das imagens de alvo e estilo para que se faça com que o histograma da imagem alvo "se pareça" com o da imagem de estilo. Para que o resultado fique visualmente bom, é interessante que usemos um espaço de cores que leva em conta a percepção humana, como o modelo conhecido como CIE-Lab (ou CIELAB). Sendo assim, vamos trabalhar com imagens convertidas para este modelo de cores.

Desta vez, não vamos implementar a conversão de imagens RGB x CIELAB, pois esta implementação tem nuances e parâmetros de calibração que a tornam um tanto delicada de se trabalhar. Ao invés disso, vamos usar uma implementação presente no submódulo `skimage.color`, que foi importado no início deste Notebook. Para converter de RGB para CIELAB, basta fazer a chamada da função `color.rgb2lab`, informando um array ou uma imagem `PIL Image` como parâmetro. Para a transformação no sentido inverso, a função se chama `color.lab2rgb`.

O modelo CIELAB tem como característica trabalhar com 3 canais diferentes. O primeiro deles, o canal L, traz informação exclusivamente de luminância da imagem, com valores $L \in [0,100]$. O segundo canal, a, traz informação de cor em um eixo que vai do verde (valores negativos) ao vermelho (valores positivos). Na implementação do módulo `skimage`, $a \in [-86.183, 98.233]$. O terceiro canal, b, traz informação de cor em um eixo que vai do azul (valores negativos) ao amarelo (valores positivos). Na implementação do módulo `skimage`, $b \in [-107.857,94.478]$.

Estes valores "quebrados" se devem aos detalhes de implementação mencionados, mas cobrem as cores perceptíveis para o ser humano. Outro detalhe importante é que, se duranto o uso de uma das funções, um *warning* do tipo: `UserWarning: Color data out of range` surgir, não se preocupe. É apenas um aviso de que algumas cores sofrerão uma "poda" para os intervalos ilustrados acima. Você pode ignorar estes avisos.

O espaço CIELAB tem como principal característica organizar as cores de acordo com a maneira que os seres humanos percebem as similaridades entre as mesmas, portanto é muito utilizado em várias aplicações onde medir a semelhança entre cores é importante.

Implemente uma função chamada `fast_transfer`, que emprega a técnica descrita a seguir.

- Extraia os arrays das imagens passadas como parâmetros, garantindo que o tipo de dado seja `np.uint8`
- Converta o array da imagem de forma para CIELAB, gerando $f$
- Converta o array da imagem de estilo para CIELAB, gerando $e$
- Em cada canal $c$, de $f$, faça:
    - $f_{c} \leftarrow (\sigma_{c}^{e}/\sigma_{c}^{f}) \cdot (f_{c} - \mu_{c}^{f}) + \mu_{c}^{e}$
    - onde:
        - $\mu_{c}^{f}$ é a média dos valores do canal $c$ em $f$
        - $\mu_{c}^{e}$ é a média dos valores do canal $c$ em $e$
        - $\sigma_{c}^{f}$ é o desvio padrão dos valores do canal $c$ em $f$
        - $\sigma_{c}^{e}$ é o desvio padrão dos valores do canal $c$ em $e$
- Converta $f$ para RGB, gerando $saida$
- Ajuste a escala da imagem resultante: $saida \leftarrow saida \cdot L_{max}$
- Retorne $saida$

Especificações da função:

- Parâmetros:
  - `imgForma`: imagem que cederá as formas dos objetos da imagem resultante, em modo **RGB**
  - `imgEstilo`: array de dados da imagem que cederá o estilo de cores da imagem resultante, em modo **RGB**

- Retorno:
  - Imagem resultante

### Complete o código da função abaixo

In [ ]:
def fast_transfer(imgForma, imgEstilo):
  ########### COMPLETE AQUI COM SEU CÓDIGO ##############
  arr_forma = np.asarray(imgForma).astype(np.uint8)
  arr_estilo = np.asarray(imgEstilo).astype(np.uint8)

  forma_lab = color.rgb2lab(arr_forma)
  estilo_lab = color.rgb2lab(arr_estilo)

  for c in range(3):
    u_fc = np.mean(forma_lab[:, :, c])
    u_ec = np.mean(estilo_lab[:, :, c])

    sigma_fc = np.std(forma_lab[:, :, c])
    sigma_ec = np.std(estilo_lab[:, :, c])

    forma_lab[:, :, c] = (sigma_ec / sigma_fc) * (forma_lab[:, :, c] - u_fc) + u_ec

  res = np.clip(color.lab2rgb(forma_lab), 0, 1) * np.max(arr_forma)
  res = res.astype(np.uint8)

  return gera_imagem(res)


#### **Testando seu código**

Execute a célula a seguir.

In [ ]:
imgEspGC = fast_transfer(imgG, imgC)
imgEspCG = fast_transfer(imgC, imgG)

imgs = [imgG, imgC, imgEspGC, imgEspCG]
legs = ['Grilo', 'Capivara', 'Grilo <-- Capivara', 'Capivara <-- Grilo']
exibe_mosaico(imgs, legs)

#### **Comparando os métodos**

In [ ]:
def exibe_comparacao(arqA, arqB):
  imgA = Image.open(arqA)
  imgB = Image.open(arqB)

  imgEspAB = especifica_histograma(imgA, imgB)
  imgEspBA = especifica_histograma(imgB, imgA)

  imgFastAB = fast_transfer(imgA, imgB)
  imgFastBA = fast_transfer(imgB, imgA)

  imgs = [imgA, imgB, imgEspAB, imgEspBA, imgFastAB, imgFastBA]
  legs = [
      arqA,
      arqB,
      'Especificação: %s <-- %s' % (arqA, arqB),
      'Especificação: %s <-- %s' % (arqB, arqA),
      'Fast Transfer: %s <-- %s' % (arqA, arqB),
      'Fast Transfer: %s <-- %s' % (arqB, arqA)
      ]
  exibe_mosaico(imgs, legs)

In [ ]:
exibe_comparacao('grilo.jpg', 'capivara.jpg')

In [ ]:
exibe_comparacao('hindu.jpg', 'favela.jpg')

In [ ]:
exibe_comparacao('antiga01.jpg', 'antiga03.jpg')

In [ ]:
exibe_comparacao('grilo.jpg', 'blue_tang.jpg')

In [ ]:
exibe_comparacao('moca_cinza.jpg', 'bh2.png')

#### **Bons estudos!**

- Lembre-se de enviar **apenas** o notebook
- **Limpe as saídas em tela** antes de enviar. Isso diminui significativamente o tamanho do arquivo final e não gera problemas de recusa de submissão no Moodle. Para isso:
  - Vá no menu *Editar*
  - Escolha a opção *Limpar todas as saídas*
  - Salve o notebook